## Command Pattern

- __Type:__ Behavioral
- __Popularity: ★★★★☆__
- __Complexity: ★★★☆☆__

### Intent:
__Command__ is a behavioral design pattern that turns a request into a stand-alone object containing all information about the request. This transformation allows you to parameterize methods with different requests, delay or queue a request's execution, and support undoable operations.

### Problem:
You need to decouple an object that invokes an operation from the provider that actually performs it. You might also need to:

- Parameterize objects with operations to be performed
- Queue operations for later execution
- Support undoing operations
- Store a history of commands for auditing or logging
- Implement callback functionality in a structured way

Without the Command pattern, systems that need these capabilities often end up with complex, tightly coupled code that's difficult to maintain and extend.

### Solution:
The Command pattern suggests encapsulating a request as an object. The key idea is to extract all the request details (the object being called, the name of the method, and the list of arguments) into a separate command class with a single method that triggers this request.

Command objects serve as links between various GUI and business logic objects. From now on, the GUI object doesn't need to know what business logic object will receive the request and how it'll be processed. The GUI object just triggers the command, which handles all the details.

This pattern introduces several components:
- **Command**: The interface that declares the execute method
- **ConcreteCommand**: Implements the Command interface and defines binding between a Receiver object and an action
- **Client**: Creates and configures concrete command objects
- **Invoker**: Asks the command to carry out the request
- **Receiver**: Knows how to perform the operation associated with the command

### Diagram:
```mermaid
classDiagram
    class Command {
        <<interface>>
        +execute()
        +undo()
    }
    
    class ConcreteCommand {
        -receiver
        +execute()
        +undo()
    }
    
    class Invoker {
        -commands
        +setCommand(command)
        +executeCommand()
        +undoCommand()
    }
    
    class Receiver {
        +action()
    }
    
    class Client {
    }
    
    Command <|-- ConcreteCommand
    ConcreteCommand o-- Receiver : has >
    Invoker o-- Command : uses >
    Client ..> ConcreteCommand : creates
    Client ..> Receiver : creates
```

In [ ]:
from abc import ABC, abstractmethod


# Command interface
class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass

    @abstractmethod
    def undo(self) -> None:
        pass

In [1]:
# Receiver class
class Editor:
    def __init__(self):
        self.text = ""

    def write(self, text):
        self.text += text
        print(f"Editor: Added text '{text}'")
        print(f"Current text: '{self.text}'")

    def delete_last(self, length):
        deleted_text = self.text[-length:]
        self.text = self.text[:-length]
        print(f"Editor: Deleted text '{deleted_text}'")
        print(f"Current text: '{self.text}'")
        return deleted_text

In [2]:
# Concrete Commands
class WriteCommand(Command):
    def __init__(self, editor: Editor, text: str):
        self.editor = editor
        self.text = text

    def execute(self) -> None:
        self.editor.write(self.text)

    def undo(self) -> None:
        self.editor.delete_last(len(self.text))


class DeleteCommand(Command):
    def __init__(self, editor: Editor, length: int):
        self.editor = editor
        self.length = length
        self.backup = ""

    def execute(self) -> None:
        self.backup = self.editor.delete_last(self.length)

    def undo(self) -> None:
        self.editor.write(self.backup)

NameError: name 'Command' is not defined

In [ ]:
# Invoker
class CommandHistory:
    def __init__(self):
        self.history = []

    def push(self, command: Command):
        self.history.append(command)

    def pop(self) -> Command:
        return self.history.pop() if self.history else None


class Application:
    def __init__(self):
        self.editor = Editor()
        self.history = CommandHistory()

    def execute_command(self, command: Command):
        command.execute()
        self.history.push(command)

    def undo(self):
        command = self.history.pop()
        if command:
            print("Undoing the last command...")
            command.undo()
        else:
            print("Nothing to undo.")

In [ ]:
# Client code
app = Application()

# Execute write command
app.execute_command(WriteCommand(app.editor, "Hello, "))

# Execute another write command
app.execute_command(WriteCommand(app.editor, "World!"))

# Execute delete command
app.execute_command(DeleteCommand(app.editor, 6))

# Undo last command (delete)
app.undo()

# Undo previous command (write "World!")
app.undo()

Editor: Added text 'Hello, '
Current text: 'Hello, '
Editor: Added text 'World!'
Current text: 'Hello, World!'
Editor: Deleted text 'World!'
Current text: 'Hello, '
Undoing the last command...
Editor: Added text 'World!'
Current text: 'Hello, World!'
Undoing the last command...
Editor: Deleted text 'World!'
Current text: 'Hello, '


### Example code with function commands:
Here's another common way to implement commands in Python using functions and callable objects:

In [ ]:
# Using functions as commands (more Pythonic approach)
from typing import Callable, Any, List


class FunctionalCommand:
    def __init__(self, execute_func: Callable, undo_func: Callable):
        self.execute_func = execute_func
        self.undo_func = undo_func

    def execute(self):
        return self.execute_func()

    def undo(self):
        return self.undo_func()


# Calculator example
class Calculator:
    def __init__(self):
        self.value = 0
        self.history: List[FunctionalCommand] = []

    def execute(self, command: FunctionalCommand):
        result = command.execute()
        self.history.append(command)
        return result

    def undo(self):
        if not self.history:
            return None
        command = self.history.pop()
        return command.undo()

    def add(self, value):
        # Create closure to capture the current state
        def execute_func():
            old_value = self.value
            self.value += value
            print(f"Added {value}, result: {self.value}")
            return old_value

        def undo_func():
            self.value -= value
            print(f"Undid add {value}, result: {self.value}")
            return self.value

        return FunctionalCommand(execute_func, undo_func)

    def multiply(self, value):
        def execute_func():
            old_value = self.value
            self.value *= value
            print(f"Multiplied by {value}, result: {self.value}")
            return old_value

        def undo_func():
            self.value /= value
            print(f"Undid multiply by {value}, result: {self.value}")
            return self.value

        return FunctionalCommand(execute_func, undo_func)


# Using the functional command pattern
calc = Calculator()
calc.execute(calc.add(5))
calc.execute(calc.multiply(2))
calc.execute(calc.add(10))
calc.undo()
calc.undo()

### Real-world analogies:

1. Restaurant Order System:

   In a restaurant, a waiter takes your order (command) written on an order slip. The waiter doesn't prepare the food but gives the order to the chef (receiver). The order slip contains all details needed to prepare the dish. The chef executes the order and the waiter doesn't need to know how the food is prepared. The order can be queued if the chef is busy, and could theoretically be "undone" if the customer changes their mind before preparation begins.

2. Remote Control:

   A TV remote control has buttons (invokers) that send specific commands to the TV (receiver). Each button press encapsulates a command (turn on/off, change channel, adjust volume). The remote doesn't need to know how the TV works internally, it just sends commands. Different devices can use the same remote if they understand the same command protocol.

### When to use:
- When you need to parameterize objects with operations
- When you want to queue operations, schedule their execution, or execute them remotely
- When you need to implement reversible operations with undo/redo functionality
- When you're building a system that needs to support transaction-like behavior
- When you want to structure a system around high-level operations built on primitive operations
- When implementing callbacks and event handler systems in a structured way

### Python-specific implementation notes:
- Python's first-class functions make implementing the Command pattern more flexible than in some other languages
- Lambda functions can be used for simple commands without creating separate command classes
- The `functools.partial` function can be useful for binding parameters to commands
- Python's built-in `cmd` module provides a framework for line-oriented command interpreters
- Many Python GUI frameworks use the Command pattern for event handling
- Python's dynamic typing allows for more flexible command implementations compared to statically typed languages

### Related patterns:
- **Chain of Responsibility**: Like Command, can be used to decouple sender and receiver, but Chain of Responsibility passes a request along a chain while Command normally specifies a sender-receiver connection
- **Memento**: Can be used alongside Command to maintain the state required for undo operations
- **Strategy**: Both patterns can be implemented with similar structures, but Strategy usually specifies how something is done, while Command specifies when and what is done
- **Factory Method**: Can be used to create commands without coupling the invoker to concrete command classes
- **Composite**: Can be used to implement macro commands (commands that consist of other commands)